# Pandas数值运算

Pandas继承了Numpy的功能

## 保留索引

Numpy的通用函数适用于Pandas的Series和DataFrame对象

In [2]:
import pandas as pd
import numpy as np

In [2]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int64

In [3]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)), columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


使用Numpy函数，声称另一个保留索引的Pandas对象

In [4]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

In [5]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


## 索引对齐

当两个Series或DataFrame对象进行二元计算，会对齐两个对象的索引。

### (1)Series索引对齐

In [3]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662, 'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193, 'New York': 19651127}, name='population')
population / area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

结果数组的索引是两个的并集，缺失会填充NaN，如下所示：

In [8]:
area.index | population.index

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

In [9]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

自定义缺失的数据

In [10]:
A.add(B, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

### (2)DataFrame索引对齐

In [11]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)), columns=list('AB'))
A

,A,B
0,1,11
1,5,1


In [12]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)), columns=list('BAC'))
B

,B,A,C
0,4,0,9
1,5,8,0
2,9,2,6


In [13]:
A + B

,A,B,C
0,1.0,15.0,NaN
1,13.0,6.0,NaN
2,NaN,NaN,NaN


将A压缩为一维数组用均值进行填充

In [14]:
fill = A.stack().mean()
A.add(B, fill_value=fill)

,A,B,C
0,1.0,15.0,13.5
1,13.0,6.0,4.5
2,6.5,13.5,10.5



| Python Operator | Pandas Method(s)                      |
|-----------------|---------------------------------------|
| ``+``           | ``add()``                             |
| ``-``           | ``sub()``, ``subtract()``             |
| ``*``           | ``mul()``, ``multiply()``             |
| ``/``           | ``truediv()``, ``div()``, ``divide()``|
| ``//``          | ``floordiv()``                        |
| ``%``           | ``mod()``                             |
| ``**``          | ``pow()``                             |


## DataFrame与Series的运算

In [15]:
A = rng.randint(10, size=(3, 4))
A

array([[3, 8, 2, 4],
       [2, 6, 4, 8],
       [6, 1, 3, 8]])

In [16]:
A - A[0]

array([[ 0,  0,  0,  0],
       [-1, -2,  2,  4],
       [ 3, -7,  1,  4]])

In [17]:
df = pd.DataFrame(A, columns=list('QRST'))
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,-1,-2,2,4
2,3,-7,1,4


按列计算

In [18]:
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,-5,0,-6,-4
1,-4,0,-2,2
2,5,0,2,7


In [19]:
halfrow = df.iloc[0, ::2]
halfrow

Q    3
S    2
Name: 0, dtype: int64

In [20]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-1.0,NaN,2.0,NaN
2,3.0,NaN,1.0,NaN


# 处理缺失值

## Pandas的缺失值

### ``None``: Python对象类型的缺失值

``None``是Python对象，不能作为任何Numpy/Pandas数组类型的缺失值，只能用于'object'数组类型

In [4]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

'object'类型容易消耗资源，而且不能进行累计操作例如：`vals1.sum()`

In [3]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
10 loops, best of 3: 78.2 ms per loop

dtype = int
100 loops, best of 3: 3.06 ms per loop



### ``NaN``: 数值类型的缺失值

``NaN`` (acronym for *Not a Number*), 按照IEEE浮点数标准设计、在任何系统中都兼容的浮点数：

In [5]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

它将会与它接触的数据同化，全变为NaN

In [6]:
1 + np.nan

nan

In [7]:
0 *  np.nan

nan

In [8]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

NumPy提供了解决方案

In [9]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

``NaN``是一种特殊的浮点类型

### NaN和None的区别

``NaN`` and ``None`` 在适当时候可以互换

In [10]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

Pandas会将没有标签值的数据类型自动转换为NA

In [11]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

In [12]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

转换规则

|Typeclass     | Conversion When Storing NAs | NA Sentinel Value      |
|--------------|-----------------------------|------------------------|
| ``floating`` | No change                   | ``np.nan``             |
| ``object``   | No change                   | ``None`` or ``np.nan`` |
| ``integer``  | Cast to ``float64``         | ``np.nan``             |
| ``boolean``  | Cast to ``object``          | ``None`` or ``np.nan`` |

Pandas中的字符串类型是用``object``类型存储的。

## 处理缺失值

- ``isnull()``: 布尔掩码标签缺失值
- ``notnull()``: 和``isnull()``相反
- ``dropna()``: 返回一个剔除缺失值的数据
- ``fillna()``: 返回一个填充缺失值的数据的copy

### (1)发现缺失值 ``isnull()`` 和 ``notnull()``

In [13]:
data = pd.Series([1, np.nan, 'hello', None])

In [14]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

就像之前的Series和DataFrame一样，还可以用布尔值进行索引

In [15]:
data[data.notnull()]

0        1
2    hello
dtype: object

The ``isnull()`` and ``notnull()`` methods produce similar Boolean results for ``DataFrame``s.

### (2)剔除缺失值``dropna()``

In [16]:
data.dropna()

0        1
2    hello
dtype: object

In [17]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


在DataFrame中，会剔除一整行或者一整列，**默认剔除一整行**

In [18]:
df.dropna()

,0,1,2
1,2.0,3.0,5


``axis=1`` 剔除一整列

In [19]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


``how``和``thresh``来设置剔除缺失值数量的阈值

In [20]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [21]:
# 全为缺失值时才剔除
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [22]:
# thresh设置非缺失值的最少数量
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


### (1)填充缺失值

In [23]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [24]:
# 通过0来填充
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [25]:
# 向前填充
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [26]:
# 向后填充
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

``DataFrame``要设置坐标轴参数

In [27]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [28]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


如果对应的前后仍是缺失值，则仍然是缺失值